In [1]:
# ladowanie modulu od unittestow
%load_ext ipython_unittest
from collections import Counter
from utils import *
import requests 

In [41]:
%%unittest_main

class MergeTests(unittest.TestCase):
    @classmethod
    def setUpClass(cls):
        reqUsers = requests.get("https://jsonplaceholder.typicode.com/users")
        reqPosts = requests.get("https://jsonplaceholder.typicode.com/posts")
        if [reqUsers.status_code, reqPosts.status_code] != [200, 200]: 
            raise Exception("Error: couldn't retrieve data from the server") 
        cls.posts = reqPosts.json()
        cls.users = reqUsers.json()
        cls.mL = mergeLists(cls.posts, cls.users, "userId")

    def testPostsNotNull(self):
        for i in self.posts:
            for j in i:
                self.assertIsNotNone(j)

    def testUsersNotNull(self):
        for i in self.users:
            for j in i:
                self.assertIsNotNone(j)

    def testMergedProperly(self):
        postKeys = len(self.posts[0].keys())
        userKeys = len(self.users[0].keys())
        self.assertEqual(len(self.mL[0].keys()), 11)

    def testIdsOfPosts(self):
        for i in range(len(self.posts)):
            self.assertEqual(self.posts[i]['id'], i+1)

    def testIdsOfUsers(self):
        for i in range(len(self.users)):
            self.assertEqual(self.users[i]['id'], i+1)

    def testMergeListIds(self):
        for i in range(len(self.mL)):
            self.assertEqual(self.mL[i]['id'], i+1)

    def testMergeListUIds(self):
        userIds = [i+1 for i in range(len(self.users))]
        for i in self.mL:
            self.assertIn(i['userId'], userIds)

    

Success

.......
----------------------------------------------------------------------
Ran 7 tests in 0.945s

OK


<unittest.runner.TextTestResult run=7 errors=0 failures=0>